In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np
from qiskit.primitives import StatevectorSampler
from qiskit_ibm_runtime import QiskitRuntimeService
import scipy as sp
import scipy.linalg



In [14]:
circuit = QuantumCircuit(2)

circuit.h(0)
circuit.cx(0,1)
circuit.cz(0,1)
circuit.draw()

┌───┐        
q_0: ┤ H ├──■───■─
     └───┘┌─┴─┐ │ 
q_1: ─────┤ X ├─■─
          └───┘

In [3]:
# 1. A quantum circuit for preparing the quantum state |000> + i |111> / √2
qc = QuantumCircuit(3)
qc.h(0) # this is used to generate superposition
qc.p(np.pi/2 , 0) # this is to add quantum phase on the qubits 
qc.cx(0,1) # 0th-qubit-Controlled-NOT gate on 1st qubit
qc.cx(1,2) # 0th-qubit-Controlled-NOT gate on 2nd qubit
qc.draw()


┌───┐┌────────┐          
q_0: ┤ H ├┤ P(π/2) ├──■───────
     └───┘└────────┘┌─┴─┐     
q_1: ───────────────┤ X ├──■──
                    └───┘┌─┴─┐
q_2: ────────────────────┤ X ├
                         └───┘

In [4]:
# 2. Add the classical output in the form of measurement of all qubits
qc_measured = qc.measure_all(inplace=False)
job = StatevectorSampler().run([qc_measured] , shots=1000)
res = job.result()
print(f" > Counts: {res[0].data["meas"].get_counts()}")

#{'000': 497, '111': 503} which is 000 50% of the time and 111 50% of the time up to statistical fluctuations.

 > Counts: {'111': 497, '000': 503}


In [ ]:
zero = np.array([[1.0],
                [0.0]]) # this gives the representation of zero state
one = np.array([[0.0],
                [1.0]])# this gives the representation of one state

# + state which is defined by |+> = 1/\sqrt{2} (|0> + |1>). 
# The factor 1/\sqrt{2} is called a normalization factor, and helps ensure the relation to a probability distribution.

# this is How we form simple single Qubit states
NormalizedState = lambda state: state / sp.linalg.norm(state)
plus = NormalizedState(zero + one) #this is the normalized plus state 
minus = NormalizedState(zero - one)  #this is the normalized minus state 

# for hadamard gate

hadamard = 1/np.sqrt(2) * np.array([[1.0,1.0],
                                    [1.0,-1.0]])

# here we are applying hadamard on zero state that will make it a ket{+} state
newState = np.dot(hadamard,zero) 



#FOR MULTIPLE QUBITS LIKE |00> AND |11> , |++>

ZeroZero = np.kron(zero,zero) #kron is sort of a tensir product that is used by mat-lab
OneOne = np.kron(one,one)
PlusPlus = np.kron(plus,plus)

#an Entangled state like {1/sqrt(2) (|00> + |11>)}
ketState = NormalizedState(ZeroZero + OneOne)

#And if we want to create a 5-qubit state, such as |100110>? We can simply chain together Kronecker products.

def Nkron(*args):
    result = np.array([[1.0]])
    for op in args:
        result = np.kron(result,op)
    return result

FiveQS = Nkron(one,zero ,zero ,one,one,zero)